<a href="https://colab.research.google.com/github/ka1shi/Fake-News-Detection/blob/master/LIAR_PLUS_DATA_testing_v1_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np 
import pandas as pd 


import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import sklearn
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend.


In [0]:
def load_data(filename):
    """
    
    Load Fake news dataset
    
    Input:
    filename : CSV file that contains data
    
    Output:
    data : Fake news data in pandas dataframe
    """
    
    data = pd.read_csv(filename, sep='\t')
    #data = pd.read_csv(filename, encoding = "ISO-8859-1")
    #data = pd.read_csv(filename, sep='\t', encoding = "ISO-8859-1", low_memory=False) 
    
    return data

In [0]:
filename1 = '/content/drive/My Drive/LIAR_PLUS_data/train'
filename2 = '/content/drive/My Drive/LIAR_PLUS_data/test'
filename3 = '/content/drive/My Drive/LIAR_PLUS_data/val'

data_train = load_data(filename1)
data_test = load_data(filename2)
data_val = load_data(filename3)

print(data_train.shape)
print(data_test.shape)
print(data_val.shape)

(10240, 15)
(1267, 15)
(1284, 15)


In [0]:
# Combining all data

frame = [data_train, data_test, data_val]
data = pd.concat(frame)

data = sklearn.utils.shuffle(data)
data.reset_index(inplace=True, drop=True) 

data.to_csv('Liar_plus_data_final.csv')

print(data.shape)

data.head()

(12791, 15)


,ID,Label,Statement,Subject,Speaker,Job,Location,Party,BT,F,HT,MT,POF,Context,Justification
0,4880.json,false,Says his 9-9-9 tax reform plan is not regressive.,taxes,herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,several public statements,Mandel said hes always held the same view on a...
1,2328.json,half-true,Charlie Crist has six different positions on O...,health-care,marco-rubio,U.S. Senator,Florida,republican,33.0,24.0,32.0,35.0,5.0,a campaign website,So we see three main positions here: Crist has...
2,1049.json,false,Palin said stimulus money for weatherization r...,"energy,stimulus",sarah-palin,NaN,Alaska,republican,9.0,19.0,9.0,6.0,6.0,an interview with Sean Hannity on Fox,Romney said it took the president 14 days befo...
3,12914.json,true,50% of Hispanic students count on Pell Grants ...,"education,federal-budget",ruben-hinojosa,U.S. House member,Texas,democrat,0.0,0.0,0.0,0.0,0.0,a tweet,"Gingrich said during the debate that ""by 2014,..."
4,6596.json,half-true,"In Massachusetts, Gov. Romney narrowed the gap...",education,jeb-bush,NaN,Florida,republican,17.0,5.0,17.0,24.0,2.0,a convention speech,"Johnson used the word ""insidious"" to describe ..."


In [0]:
data.apply(lambda x: sum(x.isnull()), axis=0)

ID                  0
Label               0
Statement           0
Subject             2
Speaker             2
Job              3567
Location         2749
Party               2
BT                  2
F                   2
HT                  2
MT                  2
POF                 2
Context           131
Justification      97
dtype: int64

In [0]:
print(data[data['F'].isnull()])

              ID  ...                                      Justification
325    1626.json  ...  Bill Clinton said, "As secretary of state, (Hi...
10679   638.json  ...  Perry says Paine wrote that the duty of a patr...

[2 rows x 15 columns]


In [0]:
# Deleting rows which have barely-true, false, half-true, mostly-true and pants-on-fire are all null 
# and even other information is missing.

data = data[data.ID != '638.json']
data = data[data.ID != '1626.json']


In [0]:
data.shape

(12789, 15)

In [0]:
#join all columns with text field 
temp = ['Statement', 'Subject', 'Speaker','Job', 'Location', 'Party', 'Context', 'Justification']

#training data
data_temp = data[temp]

data['Full_Statement'] = data_temp.apply(lambda x: ' '.join(x.map(str)), axis=1)
data.head()

,ID,Label,Statement,Subject,Speaker,Job,Location,Party,BT,F,HT,MT,POF,Context,Justification,Full_Statement
0,4880.json,false,Says his 9-9-9 tax reform plan is not regressive.,taxes,herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,several public statements,Mandel said hes always held the same view on a...,Says his 9-9-9 tax reform plan is not regressi...
1,2328.json,half-true,Charlie Crist has six different positions on O...,health-care,marco-rubio,U.S. Senator,Florida,republican,33.0,24.0,32.0,35.0,5.0,a campaign website,So we see three main positions here: Crist has...,Charlie Crist has six different positions on O...
2,1049.json,false,Palin said stimulus money for weatherization r...,"energy,stimulus",sarah-palin,NaN,Alaska,republican,9.0,19.0,9.0,6.0,6.0,an interview with Sean Hannity on Fox,Romney said it took the president 14 days befo...,Palin said stimulus money for weatherization r...
3,12914.json,true,50% of Hispanic students count on Pell Grants ...,"education,federal-budget",ruben-hinojosa,U.S. House member,Texas,democrat,0.0,0.0,0.0,0.0,0.0,a tweet,"Gingrich said during the debate that ""by 2014,...",50% of Hispanic students count on Pell Grants ...
4,6596.json,half-true,"In Massachusetts, Gov. Romney narrowed the gap...",education,jeb-bush,NaN,Florida,republican,17.0,5.0,17.0,24.0,2.0,a convention speech,"Johnson used the word ""insidious"" to describe ...","In Massachusetts, Gov. Romney narrowed the gap..."


In [0]:
# Drop all columns which we have combined earlier

#Created new dataset with data_final
data_final = data.copy()
data_final = data.drop(temp, axis=1)

In [0]:
columns = ['ID', 'BT', 'F', 'HT', 'MT', 'POF']
data_final.drop(columns, axis=1, inplace=True)

data_final.head()

,Label,Full_Statement
0,false,Says his 9-9-9 tax reform plan is not regressi...
1,half-true,Charlie Crist has six different positions on O...
2,false,Palin said stimulus money for weatherization r...
3,true,50% of Hispanic students count on Pell Grants ...
4,half-true,"In Massachusetts, Gov. Romney narrowed the gap..."


In [0]:
EMBEDDING_FILE = '/content/drive/My Drive/glove.6B/glove.6B.300d.txt'

In [0]:
data_train = data_final['Full_Statement'].str.lower()

In [0]:
data_train.head()

0    says his 9-9-9 tax reform plan is not regressi...
1    charlie crist has six different positions on o...
2    palin said stimulus money for weatherization r...
3    50% of hispanic students count on pell grants ...
4    in massachusetts, gov. romney narrowed the gap...
Name: Full_Statement, dtype: object

In [0]:
# convert categorical labels to numerical with one hot encoding
from keras.utils import np_utils

data_label = data_final["Label"].tolist()
print(data_label)

encoder = LabelEncoder()
encoder.fit(data_label)
encoded_y = encoder.transform(data_label)
print(encoded_y)
# convert to dummy variables (i.e. one hot encoded)
y_labels = np_utils.to_categorical(encoded_y, 6)

print(y_labels[0:4])

['false', 'half-true', 'false', 'true', 'half-true', 'false', 'mostly-true', 'true', 'mostly-true', 'barely-true', 'mostly-true', 'barely-true', 'half-true', 'half-true', 'mostly-true', 'pants-fire', 'true', 'half-true', 'false', 'false', 'false', 'false', 'false', 'pants-fire', 'pants-fire', 'pants-fire', 'true', 'mostly-true', 'barely-true', 'pants-fire', 'true', 'true', 'false', 'mostly-true', 'false', 'false', 'half-true', 'pants-fire', 'pants-fire', 'barely-true', 'mostly-true', 'false', 'half-true', 'half-true', 'true', 'true', 'barely-true', 'true', 'true', 'barely-true', 'pants-fire', 'pants-fire', 'mostly-true', 'false', 'mostly-true', 'barely-true', 'false', 'true', 'mostly-true', 'false', 'barely-true', 'mostly-true', 'half-true', 'half-true', 'mostly-true', 'false', 'false', 'half-true', 'pants-fire', 'half-true', 'half-true', 'false', 'false', 'half-true', 'half-true', 'mostly-true', 'true', 'barely-true', 'true', 'half-true', 'barely-true', 'true', 'mostly-true', 'true', 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data_train, y_labels, 
                                                    test_size=0.2, random_state=13)


In [0]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(10231,)
(10231, 6)
(2558,)
(2558, 6)


In [0]:
X_train.head()

11810    since the passage of the affordable care act, ...
8613     about 40 percent of hispanic homeowners have s...
12025    eighty-five percent . . . of rhode island tax ...
5713     governing magazine has estimated that in the s...
2879     the media widely overlooked comments made by f...
Name: Full_Statement, dtype: object

In [0]:
X_test.head()

12154    there have been more than 300 mass shootings i...
536      when mitt romney chose paul ryan as his vice-p...
9498     of six rhode island tax-credit programs worth ...
7774     hillary's baby bond proposal would give $5,000...
10602    on the night of the wisconsin state senate vot...
Name: Full_Statement, dtype: object

In [0]:
max_features=20000
maxlen=150
embed_size=300

In [0]:
tok=text.Tokenizer(num_words=max_features,lower=True)
tok.fit_on_texts(list(X_train)+list(X_test))
X_train=tok.texts_to_sequences(X_train)
X_test=tok.texts_to_sequences(X_test)
x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
x_test=sequence.pad_sequences(X_test,maxlen=maxlen)

In [0]:
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [0]:
word_index = tok.word_index
#prepare embedding matrix
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
sequence_input = Input(shape=(maxlen, ))
x = Embedding(max_features, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool]) 
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.1)(x)
preds = Dense(6, activation="sigmoid")(x)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 150, 300)     6000000     input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 150, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 150, 256)     329472      spati

In [0]:
batch_size = 128
epochs = 4
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.9, random_state=233)

In [0]:
#model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),callbacks = callbacks_list,verbose=1)
model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),verbose=1)
#Loading model weights
#model.load_weights(filepath)
print('Predicting....')
y_pred = model.predict(x_test,batch_size=1024,verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 9207 samples, validate on 1024 samples
Epoch 1/4
9207/9207 [==============================] - 88s 10ms/step - loss: 1.7551 - acc: 0.2098 - val_loss: 1.7533 - val_acc: 0.2051
Epoch 2/4
9207/9207 [==============================] - 85s 9ms/step - loss: 1.7219 - acc: 0.2307 - val_loss: 1.7256 - val_acc: 0.2256
Epoch 3/4
9207/9207 [==============================] - 87s 9ms/step - loss: 1.6908 - acc: 0.2508 - val_loss: 1.6945 - val_acc: 0.2627
Epoch 4/4
9207/9207 [==============================] - 87s 9ms/step - loss: 1.6485 - acc: 0.2872 - val_loss: 1.6975 - val_acc: 0.2725
Predicting....
2558/2558 [==============================] - 8s 3ms/step


In [0]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted')             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

In [0]:
print(y_test[0:4])

[[0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]]


In [0]:
y_test_encode = np.argmax(y_test, axis=1)

print(y_test_encode[0:4])

[2 1 5 1]


In [0]:
print(y_pred[0:4])

[[0.27534664 0.22430205 0.46653906 0.55066514 0.0859209  0.6223705 ]
 [0.47326207 0.5655838  0.15101412 0.18073514 0.4873682  0.08750907]
 [0.25933692 0.2302689  0.518626   0.47115806 0.07337144 0.49401316]
 [0.32970807 0.1978175  0.42330426 0.21765995 0.1754928  0.24193871]]


In [0]:
y_predict=np.argmax(y_pred, axis=1)

print(y_predict[0:4])

[5 1 2 2]


In [0]:
accuracy, precision, recall, F1 = get_metrics(y_test_encode, y_predict)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, F1 = %.3f" % (accuracy, precision, 
                                                                       recall, F1))

accuracy = 0.248, precision = 0.247, recall = 0.248, F1 = 0.242
